<a href="https://www.kaggle.com/fanbyprinciple/gpt3-notebook?scriptVersionId=84122419" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
!pip install openai

     |████████████████████████████████| 466 kB 911 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 161 kB 12.6 MB/s            
     |████████████████████████████████| 242 kB 12.3 MB/s            
  Created wheel for openai: filename=openai-0.11.5-py3-none-any.whl size=161463 sha256=e5a761f37d8c78f3d20619f07ea710d768429c85758ca29ec4a419b4820f3721
  Stored in directory: /root/.cache/pip/wheels/5a/52/66/06c234d9748d81b9622d0b9765c3bd10e99d059ed93f13b0b3
Successfully built openai


In [3]:

"""Creates the Example and GPT classes for a user to interface with the OpenAI
API."""

import openai
import uuid


def set_openai_key(key):
    """Sets OpenAI key."""
    openai.api_key = key


class Example:
    """Stores an input, output pair and formats it to prime the model."""
    def __init__(self, inp, out):
        self.input = inp
        self.output = out
        self.id = uuid.uuid4().hex

    def get_input(self):
        """Returns the input of the example."""
        return self.input

    def get_output(self):
        """Returns the intended output of the example."""
        return self.output

    def get_id(self):
        """Returns the unique ID of the example."""
        return self.id

    def as_dict(self):
        return {
            "input": self.get_input(),
            "output": self.get_output(),
            "id": self.get_id(),
        }


class GPT:
    """The main class for a user to interface with the OpenAI API.
    A user can add examples and set parameters of the API request.
    """
    def __init__(self,
                 engine='davinci',
                 temperature=0.5,
                 max_tokens=100,
                 input_prefix="input: ",
                 input_suffix="\n",
                 output_prefix="output: ",
                 output_suffix="\n\n",
                 append_output_prefix_to_query=False):
        self.examples = {}
        self.engine = engine
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.input_prefix = input_prefix
        self.input_suffix = input_suffix
        self.output_prefix = output_prefix
        self.output_suffix = output_suffix
        self.append_output_prefix_to_query = append_output_prefix_to_query
        self.stop = (output_suffix + input_prefix).strip()

    def add_example(self, ex):
        """Adds an example to the object.
        Example must be an instance of the Example class.
        """
        assert isinstance(ex, Example), "Please create an Example object."
        self.examples[ex.get_id()] = ex

    def delete_example(self, id):
        """Delete example with the specific id."""
        if id in self.examples:
            del self.examples[id]

    def get_example(self, id):
        """Get a single example."""
        return self.examples.get(id, None)

    def get_all_examples(self):
        """Returns all examples as a list of dicts."""
        return {k: v.as_dict() for k, v in self.examples.items()}

    def get_prime_text(self):
        """Formats all examples to prime the model."""
        return "".join(
            [self.format_example(ex) for ex in self.examples.values()])

    def get_engine(self):
        """Returns the engine specified for the API."""
        return self.engine

    def get_temperature(self):
        """Returns the temperature specified for the API."""
        return self.temperature

    def get_max_tokens(self):
        """Returns the max tokens specified for the API."""
        return self.max_tokens

    def craft_query(self, prompt):
        """Creates the query for the API request."""
        q = self.get_prime_text(
        ) + self.input_prefix + prompt + self.input_suffix
        if self.append_output_prefix_to_query:
            q = q + self.output_prefix

        return q

    def submit_request(self, prompt):
        """Calls the OpenAI API with the specified parameters."""
        response = openai.Completion.create(engine=self.get_engine(),
                                            prompt=self.craft_query(prompt),
                                            max_tokens=self.get_max_tokens(),
                                            temperature=self.get_temperature(),
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop=self.stop)
        return response

    def get_top_reply(self, prompt):
        """Obtains the best result as returned by the API."""
        response = self.submit_request(prompt)
        return response['choices'][0]['text']

    def format_example(self, ex):
        """Formats the input, output pair."""
        return self.input_prefix + ex.get_input(
        ) + self.input_suffix + self.output_prefix + ex.get_output(
        ) + self.output_suffix


In [5]:
import json
import openai

You can get your openai keys from here: https://beta.openai.com/account/api-keys

In [8]:
import getpass
city = getpass.getpass('Input your open ai service key:')

Input your open ai service key: ···················································


In [10]:
openai.api_key = city

In [11]:
# Application 1 - Directly use as chatbot Use-case
gpt1 = GPT(engine="davinci", temperature=0.2, max_tokens=100)
prompt1 = "How to learn data science?"
output1 = gpt1.submit_request(prompt1)
output1.choices[0].text

'\nI have been asked this question many times. I have been asked this question many times.\n\nI have been asked this question many times.\n\nI have been asked this question many times.\n\nI have been asked this question many times.\n\nI have been asked this question many times.\n\nI have been asked this question many times.\n\nI have been asked this question many times.\n\nI have been asked this question many times.\n\nI have'

In [13]:
# Application 2 - LaTex Use-case - Text to Equation
gpt2 = GPT(engine="davinci", temperature=0.5, max_tokens=100)
# Directly use as chatbot
gpt2.add_example(Example('Two plus two equals four', '2 + 2 = 4'))
gpt2.add_example(Example('The integral from zero to infinity', '\\int_0^{\\infty}'))
gpt2.add_example(Example('The gradient of x squared plus two times x with respect to x', '\\nabla_x x^2 + 2x'))
gpt2.add_example(Example('The log of two times x', '\\log{2x}'))
gpt2.add_example(Example('x squared plus y squared plus equals z squared', 'x^2 + y^2 = z^2'))

In [14]:
prompt2 = "x squared plus 2 times x"
output2 = gpt2.submit_request(prompt2)
output2.choices[0].text

'output: x^2 + 2x\n\n'

In [15]:
# Application 3 - Translator Use-case
gpt3 = GPT(engine="davinci", temperature=0.5, max_tokens=100)
gpt3.add_example(Example('What is your name?', 'quel est votre nom?'))
gpt3.add_example(Example('What are you doing?', 'Que faites-vous?'))
gpt3.add_example(Example('How are you?', 'Comment allez-vous?'))

In [17]:
prompt3 = "where are you?"
output3 = gpt3.submit_request(prompt3)
output3.choices[0].text

'output: Où êtes-vous?\n\n'

# Lets try and train this model for producing 